In [7]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import librosa

import pandas as pd

In [8]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [9]:
# Load the model.
model = hub.load('https://www.kaggle.com/models/google/surfperch/TensorFlow2/1/1')

# load the data

data = pd.read_csv('/home/a.jajodia.229/acoustic/acoustic_knowledge_discovery/perch_testing/paola.csv')

In [10]:
labels = ["anthrop", "anthrop_boat_engine", "anthrop_bomb", "anthrop_mechanical", "bioph", "bioph_cascading_saw", "bioph_chatter", "bioph_chorus", "bioph_crackle", "bioph_croak", "bioph_damselfish", "bioph_dolphin", "bioph_double_pulse", "bioph_echinidae", "bioph_epigut", "bioph_grazing", "bioph_grouper_a", "bioph_grouper_groan", "bioph_growl", "bioph_holocentrus", "bioph_knock", "bioph_knock_croak_a", "bioph_knock_croak_b", "bioph_knock_croak_c", "bioph_low_growl", "bioph_megnov", "bioph_midshipman", "bioph_mycbon", "bioph_pomamb", "bioph_pulse", "bioph_rattle", "bioph_rattle_response", "bioph_series_a", "bioph_series_b", "bioph_stridulation", "bioph_whup", "geoph", "geoph_waves"]

In [13]:
from tqdm import tqdm

result = []


for filepath in tqdm(data['filepath']):
    waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(filepath))
    if waveform.shape[0] == 0:
        continue
    waveform = tf.squeeze(waveform)
    
    waveform = librosa.resample(waveform.numpy(), orig_sr=int(sample_rate), target_sr=32000)
    
    waveforms = tf.split(
                    waveform, [160000 for _ in range(waveform.shape[0] // 160000)] + [waveform.shape[0] % 160000], axis=0
                )
    
    sounds = []
    
    for i in range(len(waveforms)-1):
        
        output = model.infer_tf(waveforms[i][np.newaxis, :])
    
        reef_labels = output['reef_label'][0]
        
        sig_mask = tf.argmax(reef_labels)
        
        sounds.append(labels[int(sig_mask)])

    
    sounds = set(sounds)
    
    result_data = {
        "filepath": filepath,
        "sounds": sounds
    }
    
    if sounds != set([]):
        result.append(result_data)
    else:
        result.append(None)
    

100%|██████████| 4884/4884 [06:47<00:00, 11.97it/s]


In [14]:
result

[{'filepath': '/home/a.jajodia.229/acoustic/local_data/coral/Paola/Non_Degraded_Reef/June 2025/20250626_085420.WAV',
  'sounds': {'bioph_cascading_saw', 'bioph_damselfish', 'bioph_low_growl'}},
 {'filepath': '/home/a.jajodia.229/acoustic/local_data/coral/Paola/Non_Degraded_Reef/February_March 2024/20240228_230000.WAV',
  'sounds': {'bioph_cascading_saw', 'bioph_croak', 'bioph_holocentrus'}},
 {'filepath': '/home/a.jajodia.229/acoustic/local_data/coral/Paola/Non_Degraded_Reef/June 2025/20250619_123518.WAV',
  'sounds': {'bioph',
   'bioph_cascading_saw',
   'bioph_damselfish',
   'bioph_low_growl'}},
 {'filepath': '/home/a.jajodia.229/acoustic/local_data/coral/Paola/Non_Degraded_Reef/April 2024/20240306_100200.WAV',
  'sounds': {'bioph_cascading_saw', 'bioph_damselfish', 'bioph_low_growl'}},
 {'filepath': '/home/a.jajodia.229/acoustic/local_data/coral/Paola/Non_Degraded_Reef/June 2025/20250619_105518.WAV',
  'sounds': {'bioph_cascading_saw', 'bioph_low_growl'}},
 {'filepath': '/home/a.j

In [29]:
pd.value_counts(data['labels'])

/tmp/ipykernel_2932570/121508271.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(data['labels'])


labels
[0 1]    4673
[1 0]     211
Name: count, dtype: int64

In [28]:
record = []

for res in result:
    if res is not None:
        for val in res['sounds']:
            record_data = {
                'filepath': res['filepath'],
                'sound': val
            }
            record.append(record_data)
pd.DataFrame.from_records(record).to_csv('surfperch_results.csv')